MOL je poslal zemljevid ovir na kolesarski poti. Zemljevid je shranjen kot seznam nizov, ki predstavljajo "vrstice": `#` predstavlja oviro, `.` pa prosto pot.

Ker smo prejšnji teden napisali program, ki sprejme opis ovir kot seznam trojk `(x0, x1, y)`, bi radi pretvorili MOLov zemljevid v takšno obliko.

## Obvezna naloga

Napiši program, ki se začne z opisom ovire v eni vrstici, na primer

```
ovire = ".##..####...##"
```

Program mora izpisati seznam **parov**, ki predstavljajo začetke in konce ovir. Za gornjo postavitev je to `[(2, 3), (6, 9), (13, 14)]`. **Ne spreglej**, da koordinate niso opisane po "Pythonovsko": prvi stolpec ima indeks 1 in obe meji sta vključeni (`(6, 9)` pokriva tudi vrstico 9, torej vsebuje 4 `#`).

Program obvezno preskusi na nizih `".##..####...##"` in `##..#...#` (ki se pretvori v `[(1, 2), (5, 5), (9, 9)]`). **Pazi: v podatkih morajo biti napisani nizi v *točno takšni* obliki. Tu ne dodajaj ničesar na začetel ali konec; če želiš, pa lahko to kasneje naredi program sam.**

### Rešitev

Da bomo preskušali vse sitnosti naenkrat, vzemimo tale niz:

In [1]:
ovire = "##...#.###..##"
#        12345678901234

Vsebuje oviro na začetku in na koncu, oviro dolžine 1 in oviri, ki sta razmaknjeni za 1. Spodaj smo napisali številke, da lažje razberemo pričakovani rezultat: `[(1, 2), (6, 6), (7, 9), (13, 14)]`.

Nalogo je možno rešiti na kup načinov. Vsem je skupno to, da se je potrebno dobro zorganizirati. Da naloga ne bi bila pretežka, sem na predavanju, namenjenem začetnikom (starejši mački pa naj se kar malo potrudijo) predlagal, da na začetek in konec niza prilepijo piko, potem pa opazujejo vse `#`, ki so takoj za ali tik pred piko. Zanimajo nas njihovi indeksi; tisti za piko predstavljajo začetke, tisti po pa konce ovir.

In [2]:
ovire = "##...#.###..##"


ovire = "." + ovire + "."
bloki = []
for i in range(len(ovire)):
    if ovire[i] == "#":
        if ovire[i - 1] == ".":
            zacetek = i
        if ovire[i + 1] == ".":
            bloki.append((zacetek, i))
print(bloki)

[(1, 2), (6, 6), (8, 10), (13, 14)]


Če vidimo `#`, pred katerim je `.`, si zapomnim, da je to začetek. Če vidimo `#`, za katerim je `.`, v seznam blokov dodamo blok od začetka do trenutnega indeksa.

Za posamične `#` se zgodi oboje naenkrat, zato tu nismo uporabili kakega `else` ali `elif`.

Stolpce v tej nalogi štejemo od `1` ne od `0`. Naši indeksi bodo pravilni, saj smo na začetek dodali `.`.

Zdaj pa izboljšave.

Na predavanjih običajno stokam, da `range(len(ovire))` ni lepo pisati - sploh, kadar potrebuješ tako indeks kot element. Raje pišimo:

In [3]:
ovire = "##...#.###..##"


ovire = "." + ovire + "."
bloki = []
for i, znak in enumerate(ovire):
    if znak == "#":
        if ovire[i - 1] == ".":
            zacetek = i
        if ovire[i + 1] == ".":
            bloki.append((zacetek, i))
print(bloki)

[(1, 2), (6, 6), (8, 10), (13, 14)]


Se lahko zgodi, da bi bil indeks `i + 1` prevelik? To bi bilo možno le, če bi bil zadnji znak `#` in bi se torej spraševali po znaku za njim. To pa ne more biti, saj smo na konec dodali piko. Prav tako `i - 1` nikoli ne bo `-1`, saj je ničti znak vedno `.` in ne `#`.

Še lepše je, če uporabimo čistokrvni `zip`. V tem primeru lahko pike dodamo kar v klicu `zip`-a.

In [4]:
ovire = "##...#.###..##"


bloki = []
for i, (prej, znak, potem) in enumerate(zip("." + ovire, ovire, ovire[1:] + "."), start=1):
    if znak == "#":
        if prej == ".":
            zacetek = i
        if potem == ".":
            bloki.append((zacetek, i))
print(bloki)

[(1, 2), (6, 6), (8, 10), (13, 14)]


Tule se je sicer zdaj zgodilo kar nekaj reči, o katerih se na predavanjih še nismo pogovarjali.

Najprej, kaj zipamo. Tole

Okrog `zip`-a pride še `enumerate`, da bomo imeli indekse; vse skupaj je potrebno razpakirati kot `for i, (prej, znak, potem)`, saj imamo pare (indeks, element), pri čemer je element trojka.

Končno, ker morajo indeksi teči od `1`, smo v `enumerate` dodali `start=1`.

Zdaj pa malo drugačna različica: zabeležimo vse spremembe.

In [5]:
ovire = "##...#.###..##"


ovire = "." + ovire + "."
spremembe = []
for i in range(len(ovire)):
    if ovire[i] != ovire[i - 1]:
        spremembe.append(i)
        
spremembe

[1, 3, 6, 7, 8, 11, 13, 15]

Te indekse je potrebno popariti:

In [6]:
list(zip(spremembe[::2], spremembe[1::2]))

[(1, 3), (6, 7), (8, 11), (13, 15)]

Žal so drugi elementi teh terk - konci blokov - preveliki. Ker terk ni mogoče spreminjati, nam (dokler se ne naučimo, kako to storiti drugače) ne preostane drugega kot prepisati jih v nov seznam. No, saj ni tako hudo.

In [7]:
ovire = "##...#.###..##"


ovire = "." + ovire + "."
spremembe = []
for i in range(len(ovire)):
    if ovire[i] != ovire[i - 1]:
        spremembe.append(i)
        
bloki = []
for zac, kon in zip(spremembe[::2], spremembe[1::2]):
    bloki.append((zac, kon - 1))
    
bloki

[(1, 2), (6, 6), (8, 10), (13, 14)]

Pa brez dodajanja pik na začetek in konec? Gre, samo malo več sitnosti je. Recimo tako:

In [8]:
ovire = "##...#.###..##"

prej = "."
bloki = []
for i, znak in enumerate(ovire, start=1):
    if znak != prej:
        if znak == "#":
            zacetek = i
        else:
            bloki.append((zacetek, i))
    prej = znak
    
if ovire[-1] == "#":
    bloki.append((zacetek, len(ovire)))
    
bloki        

[(1, 3), (6, 7), (8, 11), (13, 14)]

V spremenljivko `prej` beležimo, kakšen je bil prejšnji znak. V začetku se pretvarjamo, da je `"."`. To poskrbi, da zaznamo oviro na začetku niza. Za oviro na koncu pa poskrbimo z `if` po zanki.

Če ne vemo, da ima `enumerate` lahko tudi argument `start`, namesto `i` vedno pišemo `i + 1`.

## Dodatna naloga

Napiši program, ki prejme celotni zemljevid in izpiše seznam ovir, kot smo ga vajeni iz prejšnje naloge. Če zemljevid definiramo z

```
zemljevid = [
    "......",
    "..##..",
    ".##.#.",
    "...###",
    "###.##",
]
```

mora izpisati `[(3, 4, 2), (2, 3, 3), (5, 5, 3), (4, 6, 4), (1, 3, 5), (5, 6, 5)]`.

Pazi, tudi vrstice so oštevilčene od 1.

### Rešitev

Vzamemo katerokoli od prejšnjih rešitev in jo zapremo v zanko

```python
for y, ovire in enumerate(zemljevid, start=1):
```

ter k paru koordinat x dodamo še številko vrstice, `y`. Če ne vemo, da lahko funkciji `enumerate` podamo argument `start=2`, pa namesto `y` dodajamo `y + 1`.

In [9]:
zemljevid = [
    "......",
    "..##..",
    ".##.#.",
    "...###",
    "###.##",
]


bloki = []
for y, ovire in enumerate(zemljevid):
    ovire = "." + ovire + "."
    for i, znak in enumerate(ovire):
        if znak == "#":
            if ovire[i - 1] == ".":
                zacetek = i
            if ovire[i + 1] == ".":
                bloki.append((zacetek, i, y + 1))
print(bloki)

[(3, 4, 2), (2, 3, 3), (5, 5, 3), (4, 6, 4), (1, 3, 5), (5, 6, 5)]
